In [1]:
# List of Imports
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import geopandas as gpd
import pandana as pdna
import scipy.stats as stats
import osmnx as ox
import shutil

from ipynb.fs.full.shpToPandana import generate_pandana_store_from_shp 
from pandana.loaders import osm
from shapely.geometry import Point

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

Load All necessary Data

In [2]:
# Load Portugal BGRI11 Polygons - epsg:3763'
# polygons_bgri = gpd.read_file('./data/BGRI_2011/CONTINENTE/BGRI11_CONT.shp', dtype={'BGRI11': str})

# polygons_bgri = polygons_bgri[polygons_bgri['LUG11DESIG']!='Residual']

In [18]:
# Load Portugal Municipalities Polygons - epsg:3763'
polygons_freguesias = gpd.read_file('./data/freguesias_portugal/OgrGeoJSON.shp')

# Dissolve to get Municipalities over the inumerous polygons
dissolveByPreference='Concelho'
polygons_freguesias = polygons_freguesias.dissolve(by=dissolveByPreference,as_index=False)

In [4]:
# Load Schools Data - 'epsg:3763'
# schoolsNetwork = gpd.read_file('./data/escolas_portugal/Escolas_EB1_2019_2.shp')

# Select only the schools with primary education
# schoolsNetwork = schoolsNetwork[schoolsNetwork['ENSINOS'].str.contains('B')]

In [5]:
# polygons_bgri[polygons_bgri['DTMN11']=='0105'].plot() # 2011 - CAOP 2010

In [6]:
# polygons_freguesias[polygons_freguesias['Concelho']=='AVEIRO'].plot() # 2018 CAOP 

In [7]:
# polygons_bgri[polygons_bgri['LUG11DESIG']=='Aveiro'].plot()

Get Urban Network for each one of the BGRI

In [8]:
def create_network(polygons_dataframe, desired_network, dissolve_column='Concelho'):
    """Create HDF5 with street network graph from desired polygons from a 
    large dataframe from different places. A good use case would be a Geopandas dataframe
    with polygons from a region, or a district.
    Parameters
    ----
    dissolve_column:The column wicth will be used to fecth the results.
    desired_network:The code, name, or whatever is stored in dissolve_column.
    polygons_dataframe:Geopandas Dataframe with polygons data.
    Returns:
    ----
    None
    """
    polygonAsSeries = polygons_dataframe[polygons_dataframe[dissolve_column]==desired_network]
    if(len(polygonAsSeries)==1):
        polygonAsSeries = polygonAsSeries.squeeze()
    elif(len(polygonAsSeries)>1):
        polygons_dataframe = polygons_dataframe.dissolve(by=dissolve_column, as_index=False)
        polygonAsSeries = polygons_dataframe[polygons_dataframe[dissolve_column]==desired_network].squeeze()
        
    latlng_geom, _ = ox.project_geometry(polygonAsSeries.geometry, crs={'init':'epsg:3763'}, to_latlong=True)
    network = ox.graph_from_polygon(latlng_geom, network_type='all_private')
    # Save Shapefile
    try: 
        ox.save_graph_shapefile(network, filename=desired_network, folder='./data/temp/')
    except:
        raise Exception("It was not possible to save %s network as shapefile.")
    else:
        # Load and Convert Shapefile
        save_path = './data/networks/'+desired_network
        load_shp_path = './data/temp/'+desired_network+'/edges/edges.shp'
        path = generate_pandana_store_from_shp(save_path, load_shp_path, weights=['dist'])
        # Removing useless shp Files
        shutil.rmtree('./data/temp/'+desired_network, ignore_errors=True)
    
    

In [9]:
# create_network(polygons_freguesias, '100601') # When Using Freguesias as Dissolve

In [10]:
# create_network(polygons_freguesias,'MIRANDELA', dissolve_column='Concelho')

In [22]:
polygons_freguesias.head()

,Concelho,geometry,Dicofre,Freguesia,Distrito,TAA,AREA_EA_Ha,AREA_T_Ha,Des_Simpli,code_mun,NUTS3_2016,NUTS_nome
0,ABRANTES,"POLYGON ((7439.424 -38619.262, 7433.312 -38623...",140104,Bemposta,SANTARÉM,ÁREA PRINCIPAL,18745.35,18745.35,Bemposta,1401,16I,Médio Tejo
1,AGUIAR DA BEIRA,"POLYGON ((51668.675 114641.367, 51008.903 1141...",090114,União das freguesias de Aguiar da Beira e Coruche,GUARDA,ÁREA PRINCIPAL,4335.72,4335.72,Aguiar da Beira e Coruche,0901,16G,Viseu Dão Lafões
2,ALANDROAL,"POLYGON ((72915.444 -123799.538, 72772.137 -12...",070107,União das freguesias de Alandroal (Nossa Senho...,ÉVORA,ÁREA PRINCIPAL,25957.92,25957.92,"N. S. Conceição, S. Brás dos Matos e Juromenha",0701,187,Alentejo Central
3,ALBERGARIA-A-VELHA,"POLYGON ((-29041.318 108973.684, -29040.788 10...",010202,Alquerubim,AVEIRO,ÁREA PRINCIPAL,1535.86,1535.86,Alquerubim,0102,16D,Região de Aveiro
4,ALBUFEIRA,"MULTIPOLYGON (((-7416.080 -286724.227, -7416.6...",080106,Albufeira e Olhos de Água,FARO,ÁREA SECUNDÁRIA,0.10,4117.69,Albufeira e Olhos de Água,0801,150,Algarve


In [23]:
teste = polygons_freguesias[polygons_freguesias['Concelho']=='AGUIAR DA BEIRA']

In [26]:
from multiprocessing import Pool
from functools import partial

def main():
    iterable = polygons_freguesias.Concelho.tolist()
    dataframe = polygons_freguesias.head(12)
    func = partial(create_network, dataframe)
    agents = 6
    chunksize = 2
    with Pool(processes=agents) as pool:
        pool.map(func, iterable, chunksize=chunksize)
        pool.close()
        pool.join()

In [27]:
if __name__ == "__main__":
    main()

Overwriting existing store...
Reading shapefile with geopandas: ./data/temp/ALANDROAL/edges/edges.shp...
Converting to PT-TM06 for Portugal Projection
Calculating length to Edges
Establishing node store...
Establishing edge store for 1473 edges...
Transferring nodeids to edges...
Note: Edges are duplicated in this step, do not use the 'twoway' setting in the pandana network if using this 
             function
Graph store construction complete...
Overwriting existing store...
Reading shapefile with geopandas: ./data/temp/ALENQUER/edges/edges.shp...
Converting to PT-TM06 for Portugal Projection
Calculating length to Edges
Establishing node store...
Establishing edge store for 5755 edges...
Transferring nodeids to edges...
Note: Edges are duplicated in this step, do not use the 'twoway' setting in the pandana network if using this 
             function
Graph store construction complete...
Overwriting existing store...
Reading shapefile with geopandas: ./data/temp/ABRANTES/edges/edges.sh

ValueError: Incompatible indexer with Series

In [3]:
pool = Pool(processes=7)

In [4]:
pool.map()